In [13]:
# importing all the dependencies
%matplotlib inline
from IPython.display import display
from IPython.display import Image
import sys
import math
import numpy as np
import argparse
import os
import shutil
import scipy
import re
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib import cm, colors
from scipy import integrate
import scipy.constants as cst
import h5py
# the most important ones
from itertools import product as itp
from pythtb import *
from numpy import linalg as LA
from triqs.lattice.tight_binding import TBLattice
import sympy as sp
from sympy import *
import warnings

In [72]:
def sympyfy(tb_lat_obj, analytical = True):
    r"""
    returns the analytical form of the momentum space hamiltonian of the tight-binding model 
    from a tight-binding lattice object, by utilizing Fourier series
    
    Parameters
    ----------
    tb_lat_obj: triqs TBLattice object
        triqs tight binding object
    analytical: boolean, default = True
        a boolean which will cause the function will return an analytical Hamiltonian, when true, and 
        an numerical Hamiltonian otherwise
    
    Returns
    -------
    Hk_numerical: NumPy Array
        the hamiltonian of the tight-binding model in momentum space in numerical form
    Hk: NumPy Array
        the hamiltonian of the tight-binding model in momentum space in reduced analytical or symbolic form
    """
    I = sp.I
    # matrix from the axis directions in momentum space
    kx, ky, kz = sp.symbols("kx ky kz", real = True)
    k_space_matrix = sp.Matrix([kx, ky, kz])

    # symbolic dot product representation between the lattice unit vectors
    # and the momentum space matrix
    a1k, a2k, a3k = sp.symbols("a1k a2k a3k", real = True)
    lattice = sp.Matrix([a1k, a2k, a3k])

    # the number of orbitals involved in the electron hoppings
    num_orb = tb_lat_obj.n_orbitals

    # dictionary containing details about the hopping of the electrons
    TB_lat_obj_hops = tb_lat_obj.hoppings 

    # maximum hopping distances of electrons in each of the axial directions
    max_x, max_y, max_z = list(np.max(np.array(list(TB_lat_obj_hops.keys())), axis = 0))

    # number of cells involved in the hopping of electrons in each of the axial directions
    num_cells_x, num_cells_y, num_cells_z = [2 * max_coord + 1 for max_coord in [max_x, max_y, max_z]]

    # basis of the 5D tensor real-space Hamiltonian
    Hrij = np.zeros((num_cells_x, num_cells_y, num_cells_z, num_orb, num_orb), dtype = sp.exp)

    # looping through the hopping parameters of the electrons involved in the inter-orbital hoppings
    # key represents the cell coordinates of where the electrons hop to relative to the home unit cell
    # hopping represents the matrix with the embedded hopping amplitudes
    for key, hopping in TB_lat_obj_hops.items():
        rx, ry, rz = key
        Hrij[rx + max_x, ry + max_y, rz + max_z] = hopping

    # basis of the exponential term in the calculation of Hk
    Hexp = np.empty_like(Hrij, dtype = sp.exp)

    # perform the Fourier transform
    for xi, yi, zi in itp(range(num_cells_x), range(num_cells_y), range(num_cells_z)):
        coefficients = np.array([xi - max_x, yi - max_y, zi - max_z])
        r = lattice.dot(coefficients)
        eikr = sp.exp(-I * r)
        Hexp[xi, yi, zi, :, :] = eikr
    
    # summation over all real space axes
    Hk = np.sum(Hrij * Hexp, axis = (0, 1, 2))
    
    # rewriting the exponential terms in the analytical expression in terms of 
    for i, j in itp(range(num_orb), repeat = 2):
        Hk[i, j] = Hk[i, j].rewrite(sp.cos)

    # dealing with the numerical Hamiltonian
    # we convert it to a SymPy matrix to use the substitutions method available in SymPy
    Hk_numerical = sp.Matrix(Hk)
    TB_lat_obj_units = tb_lat_obj.units
    TB_lat_obj_units_transpose = np.transpose(TB_lat_obj_units)
    
    # obtaining the unit vectors
    a1 = TB_lat_obj_units_transpose[0]
    a2 = TB_lat_obj_units_transpose[1]
    a3 = TB_lat_obj_units_transpose[2]

    # numerical dot products between the unit vectors
    # and the momentum space matrix
    a1k_numerical = a1.dot(k_space_matrix)[0]
    a2k_numerical = a2.dot(k_space_matrix)[0]
    a3k_numerical = a3.dot(k_space_matrix)[0]
    
    # performing the numerical dot product substitutions
    Hk_numerical = Hk_numerical.subs(a1k, a1k_numerical)
    Hk_numerical = Hk_numerical.subs(a2k, a2k_numerical)
    Hk_numerical = Hk_numerical.subs(a3k, a3k_numerical)

    # converting the numerical Hamiltonian to a NumPy array from a SymPy matrix
    Hk_numerical = np.array(Hk_numerical)

    def has_complex_exponential(matrix):
        """
        Checks if a NumPy array containing SymPy elements has a complex exponential element.

        Args:
            matrix (NumPy array): The input NumPy array containing SymPy elements
        
        Returns:
            bool: True if the matrix array contains a complex exponential element, False otherwise.
        """
        for sublist in matrix:
            for element in sublist:
                if element.is_complex and element.has(exp):
                    return True
        return False
    
    def is_hermitian(matrix):
        """
        Checks if a NumPy array containing SymPy elements is hermitian

        Args:
            matrix (NumPy array): The input NumPy array containing SymPy elements
        
        Returns:
            bool: True if the matrix is a hermitian, False otherwise
        """
        n = matrix.shape[0]
        for i in range(n):
            for j in range(n):
                if matrix[i,j] != matrix[j,i].conjugate():
                    return False
        return True

    # if is_hermitian(Hk_numerical)
    
    # warning indicating when the Hamiltonian contains a complex exponential element
    if has_complex_exponential(Hk_numerical) or has_complex_exponential(Hk):
        return warnings.warn("Your expression has a complex exponential. Choosing a different unit cell could make your Hamiltonian expression real.")
    
    # returning the analytical or numerical form of the Hamiltonian
    # depending on the user's preference
    if analytical:
        return Hk
    return Hk_numerical

In [73]:
from triqs.lattice.utils import TB_from_pythTB
w90_input = w90('sccuo2 Files', 'sccuo2')
fermi_ev = 9.6082
w90_model = w90_input.model(zero_energy = fermi_ev, min_hopping_norm = 0.05, max_distance = None)
w90_triqs = TB_from_pythTB(w90_model)
print("The analytical expression \n", sympyfy(w90_triqs))
print("The numerical expression \n", sympyfy(w90_triqs, False))

The analytical expression 
 [[-1.002632*cos(a3k) - 0.124416*cos(2*a3k) + 0.0182719999999996
  -0.056436*I*sin(a1k) - 0.056434*I*sin(a1k - a3k) + 0.056436*cos(a1k) + 0.056434*cos(a1k - a3k)]
 [0.056436*I*sin(a1k) + 0.056434*I*sin(a1k - a3k) + 0.056436*cos(a1k) + 0.056434*cos(a1k - a3k)
  -0.904748*cos(a3k) - 0.128046*cos(2*a3k) - 0.130246]]
The numerical expression 
 [[-1.002632*cos(3.939404532*kz) - 0.124416*cos(7.878809064*kz) + 0.0182719999999996
  -0.056436*I*sin(1.762070113*kx + 1.762070113*ky) - 0.056434*I*sin(1.762070113*kx + 1.762070113*ky - 3.939404532*kz) + 0.056436*cos(1.762070113*kx + 1.762070113*ky) + 0.056434*cos(1.762070113*kx + 1.762070113*ky - 3.939404532*kz)]
 [0.056436*I*sin(1.762070113*kx + 1.762070113*ky) + 0.056434*I*sin(1.762070113*kx + 1.762070113*ky - 3.939404532*kz) + 0.056436*cos(1.762070113*kx + 1.762070113*ky) + 0.056434*cos(1.762070113*kx + 1.762070113*ky - 3.939404532*kz)
  -0.904748*cos(3.939404532*kz) - 0.128046*cos(7.878809064*kz) - 0.130246]]


In [69]:
# def is_hermitian(matrix):
#     mat_transp = np.transpose(matrix)
#     mat_transp_conj = np.conjugate(mat_transp)
#     return np.allclose(matrix, mat_transp_conj)
import numpy as np
import sympy as sp

def is_hermitian(matrix):
    n = matrix.shape[0]
    for i in range(n):
        for j in range(n):
            print(matrix[i,j])
            print(matrix[j,i])
            print(matrix[j,i].conjugate())
            if matrix[i,j] != matrix[j,i].conjugate():
                return False
    return True

x = sp.Symbol('x', real = True)
y = sp.Symbol('y', real = True)
# hermitian
exampleB = np.array([[1, x+1j*y, x-1j*y], 
                    [x-1j*y, x**2, y**2],
                    [x+1j*y, y**2, x**2]])
# not hermitian
exampleC = np.array([[1, x+1j*y, x-1j*y],
                     [x+1j*y, x**2, y**2],
                     [x-1j*y, y**2, x**2+1]])

print(is_hermitian(sympyfy(w90_triqs)))
# print(is_hermitian(sympyfy(w90_triqs, False)))
# print(is_hermitian(exampleB))
# print(is_hermitian(exampleC))
# exampleA = np.array([[1, 2+3j, 4-5j], 
#                     [2-3j, 5, 6-7j],
#                     [4+5j, 6+7j, 9]])

# print(is_hermitian(exampleA))
# print(is_hermitian(exampleB))

# print(is_hermitian(sympyfy(w90_triqs, False)))
# print(is_hermitian(sympyfy(w90_triqs)))

#---- trial 2---
# x, y = sp.symbols('x y')
# A = np.array([[1, x+1j*y, x-1j*y],  
#               [x-1j*y, x**2, y**2],
#               [x+1j*y, y**2, x**2]])
# print(type(A))
# A_sym = sp.Matrix(A)
# result = A_sym.is_hermitian
# bool_result = result.doit()
# print(result)
# if result == True:
#     print("Hermitian!")
# print(bool_result)
# print(type(A_sym))


# --- Trial 3 ---
# x, y = sp.symbols('x y')
# A = np.array([[1, x+1j*y, x-1j*y],  
#               [x-1j*y, x**2, y**2],
#               [x+1j*y, y**2, x**2]])
# A_sym = sp.Matrix(A)
# print(A_sym.H)
# result = A_sym.equals(A_sym.H)
# print(result)


# --- Trial 4 ---
# import numpy as np
# x, y = sp.symbols('x y')
# A = np.array([[1, x+1j*y, x-1j*y],  
#               [x-1j*y, x**2, y**2],
#               [x+1j*y, y**2, x**2]])
# print(A)
# # B = A.flatten
# # print(B)
# # print(A)
# values = [el.evalf() for el in A.flat]
# print(values)
# A = np.reshape(values, A.shape)
# A_dagger = np.conj(A).T
# is_hermitian = np.allclose(A, A_dagger)
# print(is_hermitian)


-1.002632*cos(a3k) - 0.124416*cos(2*a3k) + 0.0182719999999996
-1.002632*cos(a3k) - 0.124416*cos(2*a3k) + 0.0182719999999996
-1.002632*cos(conjugate(a3k)) - 0.124416*cos(2*conjugate(a3k)) + 0.0182719999999996
False
